<a href="https://colab.research.google.com/github/nealp1910/Big-Data/blob/main/Cloud_ETL_Sports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']='spark-3.0.3'

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
sports_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
sports_df.show(10)

In [ ]:
# Drop Null Data
sports_df = sports_df.dropna()
sports_df.show(10)

In [ ]:
# Dataset Count
sports_df.count()

In [ ]:
# Print Schema
sports_df.printSchema()

In [ ]:
# Create dataframe to match review_id_table from schema.sql file
review_id_df = sports_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(10)

In [10]:
review_id_df = review_id_df.dropDuplicates(["review_id"])

In [ ]:
from pyspark.sql.types import * 

review_id_df = review_id_df.withColumn("customer_id",review_id_df["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_df["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_df["review_date"].cast(DateType()))

review_id_df.printSchema()

In [ ]:
# Create dataframe to match products from schema.sql file
products_df = sports_df.select(["product_id", "product_title"])
products_df.show(10)

In [ ]:
products_df = products_df.dropDuplicates(["product_id"])

In [ ]:
products_df.printSchema()

In [ ]:
# Create dataframe to match customers table from schema.sql file
customers_df = sports_df.select(["customer_id"])
customers_df.show(10)

In [ ]:
customers_df = customers_df.groupBy("customer_id").count()
customers_df.orderBy("customer_id").select(["customer_id", "count"])
customers_df.show(10)

In [ ]:
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df.show(10)

In [ ]:
customers_df = customers_df.withColumn("customer_id",customers_df["customer_id"].cast(IntegerType()))\
    .withColumn("customer_count",customers_df["customer_count"].cast(IntegerType()))

customers_df.printSchema()

In [ ]:
# Create dataframe to match vine_table from schema.sql file
vine_df = sports_df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

In [20]:
vine_df = vine_df.dropDuplicates(["review_id"])

In [ ]:
vine_df = vine_df.withColumn("star_rating",vine_df["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine_df["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine_df["total_votes"].cast(IntegerType()))

vine_df.printSchema()

Postgres Setup

In [22]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://homework22bigdata.cru6fhjpgsmb.us-east-1.rds.amazonaws.com:5432/hw_22_big_data"
config = {"user":"postgres", 
          "password": "Vitesco1871!", 
          "driver":"org.postgresql.Driver"}


In [23]:
# Write DataFrame to review_id_table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
# Write dataframe to products table in RDS
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
# Write dataframe to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
# Write dataframe to vine table in RDS
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)